## Pipeline for both Models for Statistics and Data Processing
  compute:
  - standard deviation, varianz, mean, median, max, min per residue
  - the 10 most pathogenic variants per TMD-JMD region

### Loading Libraries and Datasets

In [76]:
import os 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.notebook import tqdm
import requests
import re
import time
import requests
import itertools

In [59]:
os.chdir("/Users/doma/Documents/Bachelor_Arbeit/Code")

# Set up paths relative to notebook location
project_root = Path.cwd() 

# Original Data Scores
pathway_amiss = project_root / "data" / "raw" / "AlphaMissense_csv"
pathway_esm = project_root / "data" / "processed" / "ESM1b_LLR_Normalized_Human_Nout_Proteome"

# Rank scores for both models
pathway_amiss_rank = project_root / "data" / "processed" / "AlphaMissense_rank_csv"
pathway_esm_rank = project_root / "data" / "processed" / "ESM1b_rank_csv"

# Dataset with entries
pathway_nout_proteome = project_root / "results" / "csv" / "Human_N_Out_Proteome_TMD_pathogenicity_1.csv"   # UniProt Entries
pathway_multispann_proteome = project_root / "data" / "raw" / "multipass.tsv"                               # UniProt Entries

# Load the both dataset with protein UniProt entries
human_nout = pd.read_csv(pathway_nout_proteome)
multispan_proteome = pd.read_table(pathway_multispann_proteome)


### Find missing proteins in ESM1b and AlphaMissense Datasets

In [29]:
# Check missing values in ESM1b and AlphaMissense
missing_proteins_esm = []
missing_proteins_amiss = []


for entry in human_nout['entry']:
    # Specify the pathway to each protein
    protein_pathway_amiss = os.path.join(pathway_amiss, entry + '.csv')    # Use os.path.isfile(path) returns True if file exists
    protein_pathway_esm_rank = os.path.join(pathway_esm_rank, entry + '_LLR_rank.csv')   

    # Check AlpaMissense dataset
    if not os.path.isfile(protein_pathway_amiss):
        missing_proteins_amiss.append(entry)

    # Check ESM1b dataset
    if not os.path.isfile(protein_pathway_esm_rank):
        missing_proteins_esm.append(entry)

### special case - Q99102 is too long for alphamissense prediction
missing_proteins_amiss.append("Q99102")

# Pad the shorter list with None to make both lists the same length
max_length = max(len(missing_proteins_esm), len(missing_proteins_amiss))
missing_proteins_esm.extend([None] * (max_length - len(missing_proteins_esm)))
missing_proteins_amiss.extend([None] * (max_length - len(missing_proteins_amiss)))

# Define output path
output_path = Path(project_root) / "data" / "processed" / "Missing_Entries_N_out_ESM_AM.csv"

# Check if output file already exists
if not output_path.exists():
    pd.DataFrame({
        'esm1b': missing_proteins_esm,
        'amiss': missing_proteins_amiss
    }).to_csv(output_path, index=False)
    print(f"File saved to {output_path}")
else:
    print(f"File already exists at {output_path}, skipping save.")

File saved to /Users/doma/Documents/Bachelor_Arbeit/Code/data/processed/Missing_Entries_N_out_ESM_AM.csv


In [30]:
# Check missing values in ESM1b and AlphaMissense in multispan proteome
missing_proteins_esm_multispan = []
missing_proteins_amiss_multispan = []


for entry in multispan_proteome['Entry']:
    
   # Specify the pathway to each protein
    protein_pathway_amiss = os.path.join(pathway_amiss, entry + '.csv')    # Use os.path.isfile(path) returns True if file exists
    protein_pathway_esm_rank = os.path.join(pathway_esm_rank, entry + '_LLR_rank.csv')   

    # Check AlpaMissense dataset
    if not os.path.isfile(protein_pathway_amiss):
        missing_proteins_amiss_multispan.append(entry)

    # Check ESM1b dataset
    if not os.path.isfile(protein_pathway_esm_rank):
        missing_proteins_esm_multispan.append(entry)

# Pad the shorter list with None to make both lists the same length
max_length = max(len(missing_proteins_esm_multispan), len(missing_proteins_amiss_multispan))
missing_proteins_esm_multispan.extend([None] * (max_length - len(missing_proteins_esm_multispan)))
missing_proteins_amiss_multispan.extend([None] * (max_length - len(missing_proteins_amiss_multispan)))

# Define output path
output_path = Path(project_root) / "data" / "processed" / "Missing_Entries_Multispan_ESM_AM.csv"

# Check if output file already exists
if not output_path.exists():
    pd.DataFrame({
        'esm1b': missing_proteins_esm_multispan,
        'amiss': missing_proteins_amiss_multispan
    }).to_csv(output_path, index=False)
    print(f"File saved to {output_path}")
else:
    print(f"File already exists at {output_path}, skipping save.")

File already exists at /Users/doma/Documents/Bachelor_Arbeit/Code/data/processed/Missing_Entries_Multispan_ESM_AM.csv, skipping save.


In [44]:
# Read CSV files with missing proteins 
n_out_miss = pd.read_csv(Path(project_root) / "data" / "processed" / "Missing_Entries_N_out_ESM_AM.csv")
multispan_miss = pd.read_csv(Path(project_root) / "data" / "processed" / "Missing_Entries_Multispan_ESM_AM.csv")

# Exclude the missing proteins from further processing in N-out
esm_nout_cleaned = human_nout[~human_nout['entry'].isin(n_out_miss["esm1b"])]
amiss_nout_cleaned = human_nout[~human_nout['entry'].isin(n_out_miss["amiss"])]

# Exclude the missing proteins from further processing in Multispan
esm_multispan_cleaned = multispan_proteome[~multispan_proteome['Entry'].isin(multispan_miss["esm1b"])]
amiss_multispan_cleaned = multispan_proteome[~multispan_proteome['Entry'].isin(multispan_miss["amiss"])]


# Combine missing entries from both sources
missing_any = set(n_out_miss["esm1b"]) | set(n_out_miss["amiss"])
missing_multispan_any = set(multispan_miss["esm1b"]) | set(multispan_miss["amiss"])

# Keep only proteins present in BOTH models
proteins_nout_cleaned = human_nout[~human_nout['entry'].isin(missing_any)].copy()
proteins_multispan_cleaned = multispan_proteome[~multispan_proteome['Entry'].isin(missing_multispan_any)].copy()


print(f"Number of proteins in human N-out that are also in alpha_missense is: {len(amiss_nout_cleaned)}")
print(f"Number of proteins in human N-out that are also in esm1b is: {len(esm_nout_cleaned)}")
print(f"Number of proteins in human N-out present in BOTH models: {len(proteins_nout_cleaned)}")
print("")

print(f"Number of proteins in human Multispan that are also in alpha_missense is: {len(amiss_multispan_cleaned)}")
print(f"Number of proteins in human Multispan that are also in esm1b is: {len(esm_multispan_cleaned)}")
print(f"Number of proteins in human Multispan present in BOTH models: {len(proteins_multispan_cleaned)}")


# Define output path
output_path_nout_cleaned = Path(project_root) / "data" / "processed" / "N_out_proteome_cleaned.csv"
output_path_multispan_cleaned = Path(project_root) / "data" / "processed" / "Multispan_proteome_cleaned.csv"

proteins_nout_cleaned.to_csv(output_path_nout_cleaned, index=False)
proteins_multispan_cleaned.to_csv(output_path_multispan_cleaned, index=False)

Number of proteins in human N-out that are also in alpha_missense is: 1514
Number of proteins in human N-out that are also in esm1b is: 1528
Number of proteins in human N-out present in BOTH models: 1514

Number of proteins in human Multispan that are also in alpha_missense is: 2827
Number of proteins in human Multispan that are also in esm1b is: 2824
Number of proteins in human Multispan present in BOTH models: 2822


In [66]:
def safe_get_uniprot_json(uniprot_id, retries=5, delay=3):
    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.json"
    for attempt in range(1, retries + 1):
        try:
            r = requests.get(url, timeout=10)
            r.raise_for_status()
            return r.json()
        except (requests.exceptions.SSLError, requests.exceptions.ConnectionError) as e:
            print(f"[Retry {attempt}/{retries}] SSL error for {uniprot_id}: {e}")
            time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"[Request error] {uniprot_id}: {e}")
            break
    print(f"[FAIL] Could not fetch UniProt data for {uniprot_id}")
    return None

In [67]:
def get_structural_features(uniprot_id, feature_types=None):
    if feature_types is None:
        feature_types = [
            "Region", "Site", "Helix", "Beta strand", 
            "Transmembrane", "Modified residue", "Glycosylation"
        ]
    else:
        return ("Feature Types: 'Turn', 'Glycosylation', 'Binding site', 'Motif', "
                "'Sequence conflict', 'Signal', 'Site', 'Modified residue', 'Helix', "
                "'Natural variant', 'Domain', 'Mutagenesis', 'Cross-link', 'Beta strand', "
                "'Disulfide bond', 'Alternative sequence', 'Region', 'Compositional bias', "
                "'Transmembrane', 'Topological domain', 'Peptide', 'Chain'")

    data = safe_get_uniprot_json(uniprot_id)
    if data is None:
        return [], None  # fail safely

    features = data.get('features', [])
    feature_list = []

    for feature in features:
        if feature['type'] in feature_types:
            if 'end' in feature['location']:
                start = int(feature['location']['start']['value'])
                end = int(feature['location']['end']['value'])
            else:
                start = end = int(feature['location']['start']['value'])
            feature_list.append({
                'type': feature['type'],
                'description': feature.get('description', ''),
                'start': start,
                'end': end
            })

    # sequence length with fallback
    seq_length = None
    seq_info = data.get('sequence')
    if seq_info:
        if 'length' in seq_info:
            try:
                seq_length = int(seq_info['length'])
            except (ValueError, TypeError):
                pass
        elif 'value' in seq_info:
            try:
                seq_length = len(seq_info['value'])
            except Exception:
                pass

    return feature_list, seq_length

In [68]:
# trying add juxtamembrane positions 
def map_features_to_residues_multi2(
        seq_length,
        feature_list,
        feature_types=None,
        jmd_pad=10        # ±-window around each TM span
    ):

    """
    Build a per-residue annotation frame.
    • data accessed from UniProt

    • feature_types are copied 1-to-1 into their own columns  
    • a new 'Juxtamembrane' column is filled with '1' for
      residues lying jmd_pad residues just outside any TM span
    """


    if feature_types is None:
        feature_types = ["Region", "Site", "Helix", "Beta strand",
                         "Transmembrane", "Modified residue", "Glycosylation"]

    # ➊ allocate empty strings for each feature column
    annot_dict = {ft: [''] * seq_length for ft in feature_types}
    jmd        = [''] * seq_length      # new column

    # ➋ first pass – fill normal feature columns and collect TM ranges
    tm_ranges = []                      # (start,end) tuples, 1-based
    for feat in feature_list:
        ftype = feat['type']
        if ftype not in feature_types:
            continue
        start, end = feat['start'], feat['end']
        label      = feat.get('description', '') or '1'

        for pos in range(start, end + 1):
            if 1 <= pos <= seq_length:
                idx = pos - 1
                annot_dict[ftype][idx] = (
                    f"{annot_dict[ftype][idx]},{label}" if annot_dict[ftype][idx] else label
                )

        if ftype == "Transmembrane":
            tm_ranges.append((start, end))

    # ➌ second pass – flag Juxtamembrane (±10 outside each TM core)
    for s, e in tm_ranges:
        for pos in range(max(1, s - jmd_pad), s):           # before TM
            jmd[pos - 1] = '1'
        for pos in range(e + 1, min(seq_length, e + jmd_pad) + 1):  # after TM
            jmd[pos - 1] = '1'

    # ➍ build the output DataFrame
    df = pd.DataFrame({'residue_position': range(1, seq_length + 1)})
    for ftype in feature_types:
        df[ftype] = annot_dict[ftype]
    df['Juxtamembrane'] = jmd          # <- new column

    return df

In [69]:
def annotate_top_variants(seq_len, top_df):
    annot = [''] * seq_len
    for rank, row in enumerate(top_df.itertuples(), 1):
        annot[row.residue_position - 1] += f"{rank}{row.variation},"
    annot = [a.rstrip(',') for a in annot]
    return annot

In [70]:
def matrix_transformation_df(esm1b_path, model="am_score"):
    df = pd.read_csv(esm1b_path, index_col=0)

    # Clip all values in the DataFrame to [0, 1] before reshaping
    df = df.clip(lower=0, upper=1)

    # Columns: e.g. 'M 1', 'L 2', ...
    columns = [re.match(r'([A-Z])\s+(\d+)', c) for c in df.columns]
    wt_res = [m.group(1) for m in columns]
    positions = [int(m.group(2)) for m in columns]
    tidy = []
    for col, wt, pos in zip(df.columns, wt_res, positions):
        for mut_aa in df.index:
            value = df.loc[mut_aa, col]
            if pd.isnull(value): continue
            tidy.append({
                'residue_position': pos,
                'residue': wt,
                'variation': mut_aa,
                model: value
            })
    tidy_df = pd.DataFrame(tidy)
    return tidy_df

In [72]:
# Available datasets:
n_out = pd.read_csv(output_path_nout_cleaned, usecols=["entry"])
multispan =  pd.read_csv(output_path_multispan_cleaned, usecols=["Entry"])

multispan.describe()

,Entry
count,2822
unique,2822
top,A0A087X1C5
freq,1


In [82]:
store_pathway = project_root / "data" / "processed" / "Multispan_Statistics_Rank"
missaligned = project_root / "data" / "processed" / "missaligned_proteins_multispan_rank.csv" 
skipped_proteins = []
topology = "multispan"   # "singlespan", "multispan"


for protein in tqdm(multispan["Entry"], desc="Processing proteins", unit="protein"):
     
    # Load the .csv file from protein_pathway_am/esm
    protein_pathway_am = os.path.join(pathway_amiss_rank, protein + '_rank.csv')
    protein_pathway_esm = os.path.join(pathway_esm_rank, protein + '_LLR_rank.csv')
    
    # Takes as input matrix-formatted pathogenicity scores and transforms for appropriate format for this pipeline
    df_protein_AM = matrix_transformation_df(protein_pathway_am, model="am_rank_score")
    df_protein_ESM = matrix_transformation_df(protein_pathway_esm, model="esm_rank_score")
    
    # Group by position and wild-type residue
    grouped_AM = df_protein_AM.groupby(['residue_position', 'residue'])
    grouped_ESM = df_protein_ESM.groupby(['residue_position', 'residue'])

    # Compute statistics
    stats_AM = grouped_AM['am_rank_score'].agg(['mean', 'median', 'std', 'max', 'min']).round(4).add_prefix('AM_')
    stats_ESM = grouped_ESM['esm_rank_score'].agg(['mean', 'median', 'std', 'max', 'min']).round(4).add_prefix('ESM_')

    # Merge the AM and ESM stats
    stats = stats_AM.merge(stats_ESM, left_index=True, right_index=True).reset_index()

    # Iterate over groups to find top 5 mutants above threshold
    threshold_AM = 0.7      # rank score 0.56625 for threshold 0.564
    threshold_ESM = 0.7            # rank score 0.50906 for threshold LLR -7.5

    # Ensure both groupings have same keys for consistent loop
    common_keys = grouped_AM.groups.keys() & grouped_ESM.groups.keys()

    # Prepare dictionaries to store the top pathogenic variants
    top_mutants_dict_AM = {}
    top_mutants_dict_ESM = {}

    for key in common_keys:
        group_AM = grouped_AM.get_group(key)
        group_ESM = grouped_ESM.get_group(key)

        # AM
        high_am = group_AM[group_AM['am_rank_score'] > threshold_AM]
        mutant_am = ','.join(high_am.sort_values('am_rank_score', ascending=False)['variation'].astype(str).str.upper())
        top_mutants_dict_AM[key] = mutant_am

        # ESM
        high_esm = group_ESM[group_ESM['esm_rank_score'] > threshold_ESM]
        mutant_esm = ','.join(high_esm.sort_values('esm_rank_score', ascending=False)['variation'].astype(str).str.upper())
        top_mutants_dict_ESM[key] = mutant_esm


    stats['key'] = list(zip(stats['residue_position'], stats['residue']))
    stats['AM_top_var'] = stats['key'].map(top_mutants_dict_AM).fillna('')
    stats['ESM_top_var'] = stats['key'].map(top_mutants_dict_ESM).fillna('')
    stats.drop(columns='key', inplace=True)
    stats = stats.reset_index()  # optional: flatten index
    

    ### Additionally add features from uniprot API
    # Check lengths from dataset and uniprot
    seq_length_dataset = int(stats["residue_position"].max())
    features, seq_length_uniprot = get_structural_features(protein)
    
    # Check length mismatch
    if seq_length_uniprot is not None and seq_length_dataset != seq_length_uniprot:
        print(f"[WARNING] Skipping {protein}: seq length mismatch (AM: {seq_length_dataset}, UniProt: {seq_length_uniprot})")
        skipped_proteins.append((protein, seq_length_dataset, seq_length_uniprot))
        continue  # skip to next protein
    
    df_annot = map_features_to_residues_multi2(seq_length_dataset, features)
    merged = stats.merge(df_annot, on='residue_position', how='left')


    ###################################################################################
    # Add top ten pathogenicities in TMD and separately for TMD+JMD regions in this form:
    # to residue where the most pathogenic variant is, assign 1H, 2P, 5D, 
    # Meaning the top 1. mutant is at this residue with variant H 
        # you already have the transmembrane region. 
    # Step 1: Boolean mask for TMD
    tmd_mask = merged['Transmembrane'].astype(bool)
    tmd_positions = set(merged.loc[tmd_mask, 'residue_position'])

    # Step 2: Filter variants in TMD for AM and ESM
    df_tmd_AM = df_protein_AM[df_protein_AM['residue_position'].isin(tmd_positions)]
    df_tmd_ESM = df_protein_ESM[df_protein_ESM['residue_position'].isin(tmd_positions)]

    # Step 3: Top 10 TMD variants
    N = 10
    top_tmd_AM = df_tmd_AM.nlargest(N, 'am_rank_score')
    top_tmd_ESM = df_tmd_ESM.nlargest(N, 'esm_rank_score')

    # Step 4: Annotate TMD top variants
    top_annotation_am_tmd = annotate_top_variants(seq_length_dataset, top_tmd_AM)
    annot_dict_am_tmd = {i + 1: val for i, val in enumerate(top_annotation_am_tmd)}
    merged['AM_TMD_top10'] = merged['residue_position'].map(annot_dict_am_tmd).fillna('')

    top_annotation_esm_tmd = annotate_top_variants(seq_length_dataset, top_tmd_ESM)
    annot_dict_esm_tmd = {i + 1: val for i, val in enumerate(top_annotation_esm_tmd)}
    merged['ESM_TMD_top10'] = merged['residue_position'].map(annot_dict_esm_tmd).fillna('')

    # Step 5: Only do TMD+JMD if singlespan
    if topology == "singlespan":
        # Define Juxtamembrane mask and combined region
        jmd_mask = merged['Juxtamembrane'].astype(bool)
        tmdjmd_mask = tmd_mask | jmd_mask
        tmdjmd_positions = set(merged.loc[tmdjmd_mask, 'residue_position'])

        # Filter and rank
        df_tmdjmd_AM = df_protein_AM[df_protein_AM['residue_position'].isin(tmdjmd_positions)]
        df_tmdjmd_ESM = df_protein_ESM[df_protein_ESM['residue_position'].isin(tmdjmd_positions)]

        top_tmdjmd_AM = df_tmdjmd_AM.nlargest(N, 'am_rank_score')
        top_tmdjmd_ESM = df_tmdjmd_ESM.nlargest(N, 'esm_rank_score')

        # Annotate
        top_annotation_am_tmdjmd = annotate_top_variants(seq_length_dataset, top_tmdjmd_AM)
        annot_dict_am_tmdjmd = {i + 1: val for i, val in enumerate(top_annotation_am_tmdjmd)}
        merged['AM_TMDJMD_top10'] = merged['residue_position'].map(annot_dict_am_tmdjmd).fillna('')

        top_annotation_esm_tmdjmd = annotate_top_variants(seq_length_dataset, top_tmdjmd_ESM)
        annot_dict_esm_tmdjmd = {i + 1: val for i, val in enumerate(top_annotation_esm_tmdjmd)}
        merged['ESM_TMDJMD_top10'] = merged['residue_position'].map(annot_dict_esm_tmdjmd).fillna('')

    
    elif topology == "multispan":
        # Remove unrelated columns
        merged.drop(columns=['Juxtamembrane', 'AM_TMDJMD_top10', 'ESM_TMDJMD_top10', 'AM_TMD_top10', 'ESM_TMD_top10'], inplace=True, errors='ignore')

        # Step 0: Normalize "Helical; Name=1" → "Helical"
        merged['Transmembrane'] = merged['Transmembrane'].replace(
            to_replace=r'^Helical(;.*)?$', value='Helical', regex=True)

        # Step 1: Relabel contiguous Helical regions
        helix_id = 1
        new_labels = []
        in_helix = False

        for val in merged['Transmembrane']:
            if val == 'Helical':
                if not in_helix:
                    label = f'Helical_{helix_id}'
                    helix_id += 1
                    in_helix = True
                new_labels.append(label)
            else:
                in_helix = False
                new_labels.append(val)

        merged['Transmembrane'] = new_labels

        # Step 2: Prepare annotation arrays (same size as sequence)
        am_annot = [''] * seq_length_dataset
        esm_annot = [''] * seq_length_dataset

        # Step 3: Group by helical regions
        helical_labels = sorted(set(lab for lab in new_labels if lab and lab.startswith("Helical_")))
        
        for helix_label in helical_labels:
            helix_mask = merged['Transmembrane'] == helix_label
            helix_positions = set(merged.loc[helix_mask, 'residue_position'])

            # Filter variants
            df_helix_AM = df_protein_AM[df_protein_AM['residue_position'].isin(helix_positions)]
            df_helix_ESM = df_protein_ESM[df_protein_ESM['residue_position'].isin(helix_positions)]

            top_helix_AM = df_helix_AM.nlargest(N, 'am_rank_score')
            top_helix_ESM = df_helix_ESM.nlargest(N, 'esm_rank_score')

            # Annotate to main arrays
            top_am = annotate_top_variants(seq_length_dataset, top_helix_AM)
            top_esm = annotate_top_variants(seq_length_dataset, top_helix_ESM)

            # Merge into overall annotations
            for i in range(seq_length_dataset):
                if top_am[i]:
                    am_annot[i] += top_am[i] + ','
                if top_esm[i]:
                    esm_annot[i] += top_esm[i] + ','

        # Clean trailing commas
        am_annot = [s.rstrip(',') for s in am_annot]
        esm_annot = [s.rstrip(',') for s in esm_annot]

        # Assign to merged
        merged['AM_Top10_Helix'] = merged['residue_position'].map(lambda x: am_annot[x - 1] if 0 < x <= seq_length_dataset else '')
        merged['ESM_Top10_Helix'] = merged['residue_position'].map(lambda x: esm_annot[x - 1] if 0 < x <= seq_length_dataset else '')

    ###################################################################################

    # save as csv
    output_path = os.path.join(store_pathway, f"{protein}_statistics.csv")
    merged.to_csv(output_path, index=False)


skipped_proteins = pd.DataFrame(skipped_proteins, columns=["protein_id", "seq_length_AM", "seq_length_UniProt"])
skipped_proteins.to_csv(missaligned, index=False)
    

Processing proteins:   0%|          | 0/2822 [00:00<?, ?protein/s]

[WARNING] Skipping P02708: seq length mismatch (AM: 482, UniProt: 457)
[WARNING] Skipping P03999: seq length mismatch (AM: 348, UniProt: 345)
[WARNING] Skipping P18507: seq length mismatch (AM: 467, UniProt: 475)
[WARNING] Skipping P29973: seq length mismatch (AM: 690, UniProt: 686)
[WARNING] Skipping P34998: seq length mismatch (AM: 444, UniProt: 415)
[WARNING] Skipping Q03518: seq length mismatch (AM: 808, UniProt: 748)
[WARNING] Skipping Q0D2K0: seq length mismatch (AM: 466, UniProt: 404)
[WARNING] Skipping Q13507: seq length mismatch (AM: 836, UniProt: 921)
[WARNING] Skipping Q2M3C6: seq length mismatch (AM: 531, UniProt: 523)
[WARNING] Skipping Q3KNS1: seq length mismatch (AM: 767, UniProt: 954)
[WARNING] Skipping Q8NBS3: seq length mismatch (AM: 861, UniProt: 875)
[WARNING] Skipping Q8TAX9: seq length mismatch (AM: 411, UniProt: 416)
[WARNING] Skipping Q8TDS5: seq length mismatch (AM: 423, UniProt: 384)
[WARNING] Skipping Q8WUG5: seq length mismatch (AM: 538, UniProt: 649)
[WARNI

In [44]:
# Define extraction function for getting information from UniProt for further analysis

def get_structural_features(uniprot_id, feature_types=None):

    if feature_types is None:
        feature_types = ["Region", "Site", "Helix", "Beta strand", "Signal"]
    else:
        return "Feature Types: 'Turn', 'Glycosylation', 'Binding site', 'Motif', 'Sequence conflict', 'Signal', 'Site', 'Modified residue', 'Helix', 'Natural variant', 'Domain', 'Mutagenesis', 'Cross-link', 'Beta strand', 'Disulfide bond', 'Alternative sequence', 'Region', 'Compositional bias', 'Transmembrane', 'Topological domain', 'Peptide', 'Chain'"


    url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.json"
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()
    features = data.get('features', [])
    feature_list = []


    for feature in features:
        if feature['type'] in feature_types:
            # Handle single-residue vs interval
            if 'end' in feature['location']:
                start = int(feature['location']['start']['value'])
                end = int(feature['location']['end']['value'])
            else:
                start = end = int(feature['location']['start']['value'])
            feature_list.append({
                'type': feature['type'],
                'description': feature.get('description', ''),
                'start': start,
                'end': end
            })
    return feature_list

# Example usage:
all_features = get_structural_features("P05067")
for feat in all_features:
    print(feat)

{'type': 'Region', 'description': 'GFLD subdomain', 'start': 28, 'end': 123}
{'type': 'Region', 'description': 'CuBD subdomain', 'start': 131, 'end': 189}
{'type': 'Region', 'description': 'Disordered', 'start': 194, 'end': 284}
{'type': 'Region', 'description': 'Heparin-binding', 'start': 391, 'end': 423}
{'type': 'Region', 'description': 'Heparin-binding', 'start': 491, 'end': 522}
{'type': 'Region', 'description': 'Collagen-binding', 'start': 523, 'end': 540}
{'type': 'Region', 'description': 'Interaction with PSEN1', 'start': 695, 'end': 722}
{'type': 'Region', 'description': 'Interaction with G(o)-alpha', 'start': 732, 'end': 751}
{'type': 'Region', 'description': 'Required for the interaction with KIF5B and for anterograde transport in axons', 'start': 756, 'end': 770}
{'type': 'Site', 'description': 'Required for Cu(2+) reduction', 'start': 170, 'end': 170}
{'type': 'Site', 'description': 'Cleavage; by caspases', 'start': 197, 'end': 198}
{'type': 'Site', 'description': 'Cleavag

- Was ist nur in alphamissense and esm1b und was kommt in beiden vor
wwie groß ist die schnittmenge in gesamten protein
    - wie groß in TMD, Helical, Domains...
  
- Die unterschiede bilden von den einzelnen 

- top5_var	pathog_var - comma separated DONE


- Nimm Varianten die die höchste Differenz zwischen den Modellen haben. 

- Raw daten übereinander DONE


- richtig normalizieren und clippen bei normalizierung  DONE
  - die raw scores müssen bereits auch geclippt sein, bevor Überlagerung von raw scores von beiden Models


Durchschnittswerte der Pathogenizitäten per Aminosäure: 
- von Human Proteome  DONE
- von Human N-out Proteome  DONE
  - Unterschiede? transmembrane hydrophobe aminoacids?
- Methionin wird immer über 1500 ???
- unterschiede zwischen modellen 

### Converting ESM1b to AlphaMissense format

In [62]:
df_protein_AM = matrix_transformation_df(protein_pathway_am)
df_protein_AM.head(5)

,residue_position,residue,variation,esm1b_score
0,1,M,A,0.50391
1,1,M,V,0.23940
2,1,M,L,0.35390
3,1,M,I,0.58870
4,1,M,F,0.42180
